# Introducción
### Índice de Precios al Consumidor
El índice de precios al consumidor (IPC) es un indicador que mide la variación de los precios a lo largo del tiempo y en una determinada región. Este cálculo se realiza con base en la variación del precio de un conjunto estandarizado de artículos. Dichos artículos pueden agruparse en distintas categorías, las cuales cuentan con una ponderación que cuantifica qué tanto influye ésta en el índice y qué importancia tiene este artículo o categoría en el gasto nacional.
### Estructura del IPC
Para el cálculo del IPC se usa un estándar de 441 artículos distintos, comprendiendo desde el arroz precodido hasta la compra de automóviles nuevos. Éstos artículos se cotizan en distintas fuentes y a lo largo de las 8 regiones de la república. Para el cálculo del IPC, las cotizaciones se trabajan por separado para cada una de las regiones.
Los 441 artículos de cada región pueden agruparse en 6 categorías distintas: Gasto Básico, Subgrupo, Grupo, Agrupación y División. 
 - Los 441 artículos se reúnen en 279 Gastos Básicos (Gasto genérico).
 - Los Gastos Básicos se reúnen en 120 Subgrupos (Subclases).
 - Los Subgrupos se reúnen en 84 Grupos (Clases).
 - Los Grupos se reúnen en 40 Agrupaciones (Grupos).
 - Las Agrupaciones se reúnen en 12 Divisiones (Divisiones).
 - Las Divisiones se reúnen para formar el Gasto Nacional.

Cada categoría cuenta con un código único que lo identifica. Por ejemplo, el artículo de Gas propano de 25 libras (044210101) pertenece al gasto básico del Gas propano (0442101), al subgrupo del Gas licuado (04421), al grupo del Gas (0442), a la agrupación de Electricidad , gas y otros combustibles (044) y a la división de Vivienda, agua, electricidad, gas y otros combustibles (04). Como podemos notar, el código de artículo tiene contenidos los códigos de las categorías anteriores, los primeros 5 dígitos del código de artículo son el código de subgrupo al que pertenece, los primeros 2 son la división, etc.

### Ponderaciones
Cada categoría cuenta con una ponderación de cuánto contribuye en el gasto nacional, y la cuál mide qué importancia tiene en los gastos. Por ejemplo un artículo de consumo diario, como los alimentos, persará mucho más en el gasto nacional que las compras ocasionales como autos nuevos o celulares, por lo que se ponderan más alto.
Por poner un ejemplo, tomemos el gasto básico del Arroz en la región 1, que cuenta con los artíclos: Arroz corriente de primera, Arroz corriente de segunda, Arros precocido. La ponderación del Arroz corriente de primera es 0.232 (53.88% del gasto básico del Arroz), del Arroz corriente de segunda es 0.107 (24.80%), y la del Arroz precodido es 0.092 (21.32%). Cabe recalcar que estas ponderaciones son para la región 1, en diferentes regiones estas ponderaciones y porcentajes serán distintos. Estos porcentajes varían dependiendo de diversos factores, por ejemplo qué tanto se consume y cotiza cada artículo; puesto que esto varía de región a región, las ponderaciones también cambiarán. 

Por otro lado, cada región de las 8 cuenta con su propia ponderación respecto a la república completa, esta ponderación también depende de una gran cantidad de parámetros y variaciones demográficas y económicas, un ejemplo de ello puede ser la población de cada región, o su actividad económica. En concreto, la región metropolitana cuenta con una ponderación de 37.35% respecto al gasto nacional, y la región de suroccidente está ponderada con el 18.53%.



# Cálculo del IPC

Ahora elaboraremos en los cálculos del Índice de Precios al Consumidor, para realizar este cálculo haremos uso de paquetes de programación que nos permitan consultar la información de las bases de datos, y manejarlos de forma eficiente.

In [334]:
#Se importan los paquetes necesarios para los cálculos.
import pandas as pd #manejo de datos
import numpy as np #parquete de matemática
import scipy.stats as sp #paquete de estadísticas
from sqlalchemy import create_engine #consultas a sql
import sqlalchemy #manejo de sql
import sys #manejo del sistema para recibir argumentos
import warnings #
from sqlalchemy import text
warnings.filterwarnings("ignore")
import pyodbc

import funciones

A continuación debemos establecer una conección con el servidor en el que se almacenan las boletas de cotización y las ponderaciones e información pertinente.

In [335]:
#Conexión a la base de datos
server = '10.0.0.153'
user = "testipc2"
pas = "Abc$2020"
#user = input('Ingrese el usuario: ')
#pas = input('Ingrese la contraseña: ')
try:
    #driver = 'SQL Server Native Client 11.0'
    driver = 'ODBC Driver 17 for SQL Server'
    database_connection = f'mssql://{user}:{pas}@{server}/master?driver={driver}'
    engine = create_engine(database_connection)
    connection = engine.connect()
    print('Conexión exitosa')
except:
    print('Fallo en la conexión')


Conexión exitosa


Ingresamos los datos que vamos a usar, año mes y década. Para todo el mes usamos década = 0, para las primeras dos décadas (década 1 y 2) usamos década = 12, para las últimas ds décadas usamos 23, y para cada década individual usamos el número de dicha semana



In [336]:
#Recibimos el año, mes y década. Deben ingresarse el número de año, mes y década. Las décadas están codificadas de la siguiente forma:
#Todo el mes: década = 0
#Las primeras dos décadas: década = 12
#Las últimas dos décadas: década = 23
#Para cada década individual se ingresa el número de década
anio = int(input('Ingrese el año: '))
mes = int(input('Ingrese el mes: '))
dec = int(input('Ingrese la década (ingrese 0 para obtener todo el mes): '))

In [337]:
#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Codigos(connect):
    querycod = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    
    return pd.read_sql(querycod, connect, index_col = 'ArtCod')

codigos = Codigos(connection)

In [338]:
#Se define una función para hallar cualquier conjunto de códigos.
#le asocia a la categoría su número en la base de datos
di = {'Art':6,'Gba':5,'Sub':4,'Gru':3,'Agr':2,'Div':1}
#le asocia a cada categoría la longitud de su código
leng = {'Div' : 2, 'Agr' : 3, 'Gru' : 4, 'Sub' : 5, 'Gba' : 7, 'Art': 9}

#Se seleccionan los códigos de los artículos, los demás códigos pueden obtenerse a partir de éstos.
def Nombres(datos, cod : str, connect = connection, dict = di):
    ''' 
    Esta función retorna un dataframe con los nombres y códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    if cod == 'Art':
        querycod = text(f"SELECT DISTINCT ArtCod, ArtNom FROM IPC2010_RN.dbo.IPC007")
    else:
        querycod = text(f'SELECT DISTINCT {cod}Cod, {cod}Nom FROM IPC2010_RN.dbo.IPCM0{dict[cod]}')

    nombres = pd.read_sql(querycod, connect, index_col = cod+'Cod')
    return pd.merge(nombres.reset_index(), datos.reset_index(), on = cod+'Cod',  how = 'inner').set_index(['RegCod',cod+'Cod'])

def CodigosCat(ind :str, cod, len = leng):
    ''' 
    Esta función retorna un dataframe con los códigos de una categoría específica.

    Args:
    cod (DataFrame) : El dataframe con los códigos de una categoría dada, puede ser de artículos, gasto báscico, división, etc
    ind (str) : El código de categoría que deseamos. Gba : Gasto Básico, Sub : Subgrupo, Gru : Grupo, Agr : Agrupación, Div : División
    '''
    return pd.DataFrame([x[:len[ind]] for x in cod.index],columns=[ind+'Cod']).drop_duplicates().set_index(ind+'Cod')

Para calcular la variación que tuvo cada artículo en el precio utilizamos todas las boletas de cotización que se realizaron en el mes a nivel nacional. Para cada fuente y artículo que se cotiza, se cuenta con una boleta en la que se ingresa la información. Los datos que necesitamos de la boleta son: La región en la que se cotizó, el artículo que se cotizó, el precio anterior registrado, el precio actual que se cotizó, la centidad anterior registrada y la cantidad actual. La cantidad es la medida en la que se cotiza, por ejemplo la leche o el agua se miden en mililítros, litros o galones; pero el arroz o frijol se miden en gramos, libras o kilos. Estas medidas son la cantdiad que se registra, esta información es importante para saber si la variación de precio se debe a un cambio verdadero o únicamente a una variación en la presentación o cantidad de venta.

In [339]:
#'''
#Aquí se mandan a traer las boletas del mes
def Boletas(anio : int, mes : int, connect):

    """Esta función retorna un dataframe con las boletas del mes correspondiente con el índice como . 
    Incluyen código de región, de artículo

    args:
    anio (int): el año de las boletas a traer
    mes (int): el mes de las boletas a traer
    connect (Connection) : conexión a la base de datos

    """
    subquery = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    query = text(f"SELECT * FROM (\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_01_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_02_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_03_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_04_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_05_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_06_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_07_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            UNION ALL\
            SELECT RegCod, ArtCod, BolNum, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem\
            FROM IPC2010_08_RN.dbo.IPC104\
            WHERE PerAno = {anio} AND PerMes = {mes} \
            AND ArtCod IN ({subquery})\
            ) I\
            ORDER BY RegCod, ArtCod")
    return pd.read_sql(query, connect, index_col=['RegCod', 'ArtCod', 'BolNum']).sort_index()

boletas = Boletas(anio, mes, connection)
#'''

In [391]:
boletas

ArtPhi   ArtPac  UraChi  UreCan ArtCR  PerSem
RegCod ArtCod    BolNum                                                
01     011110101 313510     7.00     7.00   400.0   400.0     N       1
                 313518     6.85     7.10   400.0   400.0     N       1
                 313519     7.10     7.10   400.0   400.0     N       1
                 313520     7.15     7.15   400.0   400.0     N       1
                 313523     6.95     7.10   400.0   400.0     N       1
...                          ...      ...     ...     ...   ...     ...
08     125110301 109391    75.00    75.00     1.0     1.0     N       1
                 109411    50.00    50.00     1.0     1.0     N       1
                 109424    50.00    50.00     1.0     1.0     N       1
       125110401 109340  2500.00  2500.00     1.0     1.0     N       1
                 110031  3800.00  3800.00     1.0     1.0     N       3

[48789 rows x 6 columns]

In [341]:
'''
#aqui se empieza la parte de la conexion con el otro server
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'
#'''

"\n#aqui se empieza la parte de la conexion con el otro server\n#Almacenamos la información del servidor para poder realizar la conexion más adelante\nserver = 'ipcprod.database.windows.net'\nuser = 'ipcreader'\npassword = '1pc/*2023'\ndatabase = 'db-indices'\n#"

In [342]:
'''
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)
#'''

"\n#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente\nconn_str = (\n    f'Driver={{ODBC Driver 17 for SQL Server}};'\n    f'Server={server};'\n    f'Database={database};'\n    f'Uid={user};'\n    f'Pwd={password};'\n)\n#"

In [343]:
'''
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    cursor.execute("EXEC [dbo].[sp_get_precios_recolectados_mes] 2024, 1")
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")
#'''

'\ntry:\n    conn = pyodbc.connect(conn_str)\n    cursor = conn.cursor()\n    \n    cursor.execute("EXEC [dbo].[sp_get_precios_recolectados_mes] 2024, 1")\n    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])\n    \n    conn.close()\n    print("Connection successful!")\nexcept Exception as e:\n    print(f"Error: {e}")\n#'

In [344]:
'''
boletas['codigo_articulo'] = boletas['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones
boletas['decada'] = boletas['decada'].replace(funciones.decadas)
boletas['nt_tipo'] = boletas['nt_tipo'].replace({351761:'S', 351761:'S'})
boletas['nt_tipo'] = boletas['nt_tipo'].fillna('N')
boletas = boletas[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]
boletas.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR', 'decada':'PerSem'}, inplace=True)
boletas = boletas[boletas['ArtCod'].str.len() == 9]
boletas.set_index(['RegCod', 'ArtCod', 'BolNum'], inplace=True)
boletas.sort_index(inplace=True)

boletas
#'''

"\nboletas['codigo_articulo'] = boletas['codigo_articulo'].replace(funciones.recodificacion) #esta parte hace la recodificacion con el diccionario de los codigos en el archivo funciones\nboletas['decada'] = boletas['decada'].replace(funciones.decadas)\nboletas['nt_tipo'] = boletas['nt_tipo'].replace({351761:'S', 351761:'S'})\nboletas['nt_tipo'] = boletas['nt_tipo'].fillna('N')\nboletas = boletas[['region', 'codigo_articulo', 'ine_poll_id', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo', 'decada']]\nboletas.rename(columns={'region':'RegCod', 'codigo_articulo':'ArtCod', 'ine_poll_id':'BolNum', 'cantidad_anterior':'UraChi', 'cantidad_actual':'UreCan', 'precio_anterior':'ArtPhi', 'precio_actual':'ArtPac', 'nt_tipo':'ArtCR', 'decada':'PerSem'}, inplace=True)\nboletas = boletas[boletas['ArtCod'].str.len() == 9]\nboletas.set_index(['RegCod', 'ArtCod', 'BolNum'], inplace=True)\nboletas.sort_index(inplace=True)\n\nboletas\n#"

## OUTLIERS
A continuación leemos el excel con las boletas de Outliers

In [345]:
ignorar = False
if ignorar:
    ig = pd.read_excel(f'IGNORAR{anio}-{mes}.xlsx', dtype={f'ArtCod':str,f'RegCod':str})
    ig = ig.set_index(['RegCod','ArtCod','BolNum'])
else:
    ig = []

Para calcular la variación de un producto usamos el concepto de precio en medida base $PMB$, que es el precio del artículo en unidad en medida base $UMB$, esta unidad base es una cantidad estándar sobre la cual se calcularán los precios. La fórmula para el $PMB$ es 
$$ 
PMB = \frac{P}{Q} \cdot UMB,
$$
donde $P$ es el precio cotizado, $Q$ es la cantidad cotizada, y $UMB$ es la unidad estandarizada.

Con esto podemos calcular la variación $V_{v,f}$ del precio de un artículo (o variedad) $v$ para una fuente dada $f$. La fórmula para este cálculo es 
$$
V_{v,f} = \frac{PMB_{ac}}{PMB_{hi}} = \frac{P_{ac}}{Q_{ac}}\cdot \frac{Q_{hi}}{P_{hi}},
$$
en el que $P_{ac}$ y $P_{hi}$ son el precio actual y anterior, y $Q_{ac}, Q_{hi}$ son las cantidades actual e histórica.

In [346]:
#Funcion para calcular las variaciones de cada cotización

def Variaciones(boleta, decada, outliers, ignorar = False):
    ''' 
    Esta función calcula las variaciones de la decada ingresada dado un arreglo de boletas y una decada
    
    Argumentos:
    boletas (DataFrame) : Un dataframe de las cotizaciones con las columnas RegCod, ArtCod, ArtPhi,ArtPac,UraChi,UreCan,ArtCR, PerSem y el multiíndice (RegCod, ArtCod)
    dec (int) : La década en la que se va a calcular el índice, 

    '''
    boletas = boleta.copy()
    if decada == 0:
        semana = [1,2,3]
    elif decada == 12:
        semana = [1,2]
    elif decada == 23:
        semana =[2,3]
    else:
        semana = [decada]

    boletas = boletas.loc[boletas['PerSem'].isin(semana)]
    
    Variacion = pd.DataFrame((boletas['ArtPac']*boletas['UraChi'])/(boletas['ArtPhi']*boletas['UreCan']), columns = ['Var'])
    Variacion = Variacion.replace([np.nan,0,np.inf],1)
    Variacion.loc[boletas['ArtCR'] == 'S'] = 1.0
    if ignorar:
            Variacion.loc[Variacion.index.isin(outliers.index), 'Var'] = 1.0
    return Variacion

variaciones = Variaciones(boletas, dec, ig, ignorar)



Luego, la variación acumulada $V_v$ de una variedad o artículo $v$ está dado por 
$$
V_v = \sqrt[m]{
    \sqrt[n]{\prod_{f = 1}^{n}V_{v,f}}
}
$$
con $n$ el número de fuentes cotizadas de la variedad, y $m$ es la periodicidad con la que se cotiza el artículo. La expresión dentro de la raíz $m$-ésima es la raíz $n$ de la multiplicación de las $n$ variaciones de un artículo fijo, esto es una media geométrica; la media geométrica es una forma de tomar promedios que es menos sensible a anomalías, por lo que da una medida más uniforme. 

La raíz $m$ es para tomar en cuenta cada cuánto se cotiza dicho artículo, si se cotiza cada 3 meses entonces se usa la raíz cúbica para tomar en cuenta esa periodicidad. La periodicidad de un artículo es cada cuántos meses se cotiza, por ejemplo los artículos semestrales tienen periodicidad $m=6$, y los bimestrales tienen $m=2$. También podemos ver esta cantidad como el índice relativo de cada artículo.

In [347]:
#Función que selecciona las periodicidades de los artículos, convierte el código de la periodicidad al número de meses que significa.
def Periodos(connect):
        '''
        Llama las periodicidades de todos los artículos
        Args:
        connect (Connection) : conexión a la base de datos
        '''
        subquery = text(f"SELECT DISTINCT ArtCod FROM IPC2010_01_RN.dbo.IPCPH6 \
                WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
        query = text(f"SELECT ArtCod, \
                CASE \
                WHEN ArtPRC = 5 THEN  6\
                WHEN ArtPRC = 6 THEN  12\
                WHEN ArtPRC = 1 THEN  1\
                WHEN ArtPRC = 2 THEN  2\
                WHEN ArtPRC = 3 THEN  3\
                WHEN ArtPRC = 4 THEN  4\
                ELSE 1\
                END AS ArtPRC\
                FROM IPC2010_RN.dbo.IPC007 \
                WHERE ArtCod IN \
                ({subquery})\
                ORDER BY ArtCod")
        indice = 'ArtCod'
        return pd.read_sql(query, connect, index_col=indice)


In [348]:
#Función para calcular las variaciones acumuladas de los 441 artículos
def VarFlex(var,per, cod):
    ''' 
    Esta función calcula los índices relativos (variación acumulada) de los 441 artículos para todas las regiones y retorna un arreglo con índices (region, artículo). A los artículos que no fueron cotizados les asigna variación 1

    Args
    var (DataFrame) : un dataframe con las variaciones de las fuentes con multiíndice RegCod, ArtCod
    per (DataFrame) : dataframe con las periodicidades de los artículos con índice ArtCod
    cod (DataFrame) : un dataframe con los 441 códigos de los artículos
    '''
    agrupacion = var.groupby(level = ['RegCod','ArtCod']).agg(sp.gmean)
    periodicidad = agrupacion.join(per.astype(float), how = 'outer')
    raiz = pd.DataFrame(periodicidad['Var']**(1/(periodicidad['ArtPRC'])), columns = ['Var']).sort_index()
    arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
    Codigos = cod.copy()
    Codigos['ArtCod'] = Codigos.index
    codigos2 = Codigos.merge(arreglo, how = 'cross')
    codigos2 = codigos2.set_index(['RegCod','ArtCod'])
    codigos2['Var'] = raiz['Var']
    codigos2 = codigos2.replace(np.nan, 1)
    return codigos2


In [349]:
#Llamamos las periodicidades y calculamos la variación de artículos
periodos = Periodos(connection)
ArtInd = VarFlex(variaciones, periodos, codigos)

In [350]:
#ArtInd.to_excel('indices_articulos_enero_2023.xlsx')

Los artículos de formmación nacional son una lista de artículos que no se cotizan en regiones distintas a la metropolitana pues el precio no varía de región a región, pero sí se utilizan en el cálculo del IPC. Por este motivo, al resto de regiones se les asigna el valor de la variación obtenida en la región 1.

In [351]:
#Se agregan las variaciones del armado nacional
def FormacionNacional(indart):
    ''' 
    Esta función le asigna a los productos del resto de regiones, la variación de la región metropolitana si están en el armado nacional.

    Args:
    indart (DataFrame) : dataframe de las variaciones de los artículos
    '''    
    lista = ['071110101','071110102','071110103','072420201','073210101','083110101','083110202','083110301','083110302',\
                    '083110401','093210201','093310101','095110201','123110101','123110102','123110103','123110104','124110101','125110101','125110501']
    indices = indart.copy()
    for index in lista:
        try:
            indices.loc[(indart.index.get_level_values('ArtCod') == index),'Var'] = indart.loc['01', index]['Var']
        except KeyError:
            pass
    return indices

ArtInd = FormacionNacional(ArtInd)

In [352]:
ArtInd = ArtInd.sort_index()
ArtInd

#ArtInd[ArtInd['Var'] != 1].sum()

Var
RegCod ArtCod             
01     011110101  1.004978
       011110102  1.000752
       011110103  1.016912
       011120101  0.995982
       011120102  0.988430
...                    ...
08     125110101  0.991258
       125110201  0.984105
       125110301  1.000000
       125110401  1.000000
       125110501  1.000000

[3528 rows x 1 columns]

Ahora calcularemos la variación $V_{GB}$ de un gasto básico $GB$, donde estas variaciones también cambian de región a región. La fórmula para hacer este cálculo está dada por 
$$
V_{GB} = \frac{1}{\sum_{v \in Gba}\alpha_v}\sum_{v \in Gba}\alpha_v V_v,
$$
donde las sumas son sobre todas las variedades $v$ que pertenecen al gasto básico que deseamos $Gba$; es decir, sumamos todos los artículos del gasto básico. Las ponderaciones $\alpha_v$ son las ponderaciones de los artículos respecto a su gasto básico y $v_v$ son las variaciones acumuladas de cada artículo.

Puesto que la suma de los artículos respecto a su gasto básico es 100, podemos escribir la fórmula como
$$
V_{GB} = \frac{1}{100}\sum_{v = 1}^{n}\alpha_v V_v.
$$

Una vez más, como las ponderaciones y las variaciones de artículo dependen de la región, entonces las variaciones de gasto básico también dependen de la región.

In [353]:
#Función para llamar la ponderación de cualquier categoría
def CatPon(cod : str, dic, connect):
    """Consulta las ponderaciones de los códigos de una categoría

    Se extraen las ponderaciones de las 8 regiones

    Args:
        cod (str): código de la categoría (no es un código de artículo)
        dic (dict) : diccionario de categorías a bases de datos
    """
    if cod == 'Art':
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCPH6 \
                    WHERE PerAno = 2010 AND PerMes = 12 AND ArtCod != 091110301")
    else:
        subquery = text(f"SELECT DISTINCT {cod}Cod FROM IPC2010_01_RN.dbo.IPCM0{dic[cod]}")
    
    query = text(f'SELECT RegCod, {cod}Cod, {cod}Pon\
        FROM  IPC2010_RN.dbo.IPCP0{dic[cod]}\
        WHERE {cod}Cod IN ({subquery}) \
        ORDER BY RegCod, {cod}Cod')              
    return pd.read_sql(query, connect, index_col=['RegCod', cod+'Cod'])


#Función de cálculo de índices (relativos o absolutos)
def CatInd(cod :str, indices, pon, len = leng, codigo = codigos):
    ''' 
    Recibe un dataframe de índices una categoría dada y retorna índices de una categoría superior. Si queremos calcular alguna categoría superior Gasto Básico necesitamos índices y ponderaciones de al menos Gba 

    Args:
    cod (str) : cóigo de la categoría de la que calcularemos los índices
    indices (DataFrame) : dataframe con los índices, debe tener multiíndice de la forma RegPon,{cod}Cod, y un único campo con los índices
    pon (DataFram) : Un dataframe con las ponderaciones de los índices (es decir, si indices es de Gba, las ponderaciones deben ser de Gba)
    len (dit) : el diccionario de longitudes de los códigos de cada categoría
    '''

    pond = pon.copy()
    pond.columns = ['Pon']
    index = indices.copy()
    index.columns = ['Ind']
    pond[cod+'Cod'] = pond.index.get_level_values(1).map(lambda x: x[:len[cod]])
    pond = pond.set_index([pond.index.get_level_values(0), pond[cod+'Cod'], pond.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    pond[cod+'Pon'] = pond.groupby(level = ['RegCod', cod+'Cod']).sum()
    pond[cod+'Pon']=pond[cod+'Pon'].replace(0,1)
    cont = pd.DataFrame(index['Ind']*pond['Pon']/pond[cod+'Pon'], columns = [cod+'Ind'])
    cont[cod+'Cod'] = cont.index.get_level_values(1).map(lambda x: x[:len[cod]])
    cont = cont.set_index([cont.index.get_level_values(0), cont[cod+'Cod'], cont.index.get_level_values(1)]).drop(columns= [cod+'Cod'])
    catind = cont.groupby(level = ['RegCod', cod+'Cod']).sum()

    if cod == 'Gba':
        arreglo = pd.DataFrame(['01','02','03','04','05','06','07','08'], columns= ['RegCod'])
        cods = CodigosCat(cod, codigo)
        Codigos = cods.copy()
        Codigos[cod+'Cod'] = Codigos.index
        codigos2 = Codigos.merge(arreglo, how = 'cross')
        codigos2 = codigos2.set_index(['RegCod',cod+'Cod'])
        codigos2[cod+'Ind'] = catind[cod+'Ind']
        codigos2 = codigos2.replace(np.nan, 1)
        return codigos2
    else:
        return catind
    

In [354]:
CatPon('Gba',di,connection)
#.to_excel('GBA_pond01.xlsx', merge_cells=False)

GbaPon
RegCod GbaCod          
00     0111101  0.63244
       0111201  0.17119
       0111202  1.33671
       0111203  0.93761
       0111301  2.61053
...                 ...
08     1251101  0.11386
       1251102  0.00713
       1251103  0.05303
       1251104  0.16740
       1251105  0.00000

[2511 rows x 1 columns]

In [355]:
#Calculamos los índices relativos (variación) de gasto básico de las 8 regiones
artpon = CatPon('Art', di, connection)

vargba = CatInd('Gba', ArtInd, artpon, leng)

#artpon
vargba

,,GbaInd
RegCod,GbaCod,
01,0111101,1.006474
02,0111101,1.001834
03,0111101,1.004381
04,0111101,0.999575
05,0111101,1.001413
...,...,...
04,1251105,1.000000
05,1251105,0.000000
06,1251105,1.000000


Una vez tenemos la variación de un gasto básico, podemos calcular el índice de gasto básico $I_{GB}$
$$
I_{GB} =  V_{GB}\cdot I_{GB,hi}.
$$
donde $I_{GB,hi}$ es el índice del gasto básico del mes anterior. Recordemos que este índice varía de región a región.

In [356]:
#Función para extraer índices anteriores de cualquier categoría
def IndAnt(anio : int, mes : int, cod : str, conectar, dic = di, ano = False):
    """Consulta los índices del mes anterior desde la base de datos por año, mes y categoría

    Los índices obtenidos son de un mes anterior al mes en que se calcula el IPC

    Args:
        anio (int): año actual
        mes (int): número de mes actual
        cod (str): código de categoría
    """
    if ano:
        mesant = mes
        anioant = anio - 1
    else:
        if mes == 1:
            anioant = anio -1
            mesant = 12
        else:
            anioant = anio
            mesant = mes-1

    query = text(f'SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_01_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_02_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_03_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_04_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_05_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_06_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_07_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        UNION ALL\
        SELECT RegCod, {cod}Cod, {cod}Ind\
        FROM IPC2010_08_RN.dbo.IPCPH{dic[cod]} WHERE PerAno = {anioant} AND PerMes = {mesant} AND PerSem = 3 \
        ORDER BY RegCod, {cod}Cod')
    return pd.read_sql(query, conectar, index_col=['RegCod',cod+'Cod'])

In [357]:
gbaant = IndAnt(2023, 12, 'Gba', connection)
gbaant

GbaInd
RegCod GbaCod             
01     0111101  176.653306
       0111201  245.422872
       0111202  239.854923
       0111203  154.619007
       0111301  175.033469
...                    ...
08     1241101   99.220531
       1251101  120.596493
       1251102  113.619816
       1251103  114.954121
       1251104  117.797094

[2185 rows x 1 columns]

In [358]:
gbaano = IndAnt(2023, 12, 'Gba', connection, ano = True)
gbaano

GbaInd
RegCod GbaCod             
01     0111101  159.062328
       0111201  221.200165
       0111202  233.727593
       0111203  139.351841
       0111301  174.016387
...                    ...
08     1241101   95.887042
       1251101  120.596493
       1251102  115.187051
       1251103  113.873077
       1251104  117.294604

[2185 rows x 1 columns]

In [359]:
gbaind = pd.DataFrame(gbaant['GbaInd']*vargba['GbaInd'], columns= ['GbaInd']).replace(np.nan, 0)
gbaind

GbaInd
RegCod GbaCod             
01     0111101  177.796985
       0111201  244.137471
       0111202  238.104250
       0111203  150.948101
       0111301  175.782902
...                    ...
08     1251101  119.542285
       1251102  111.813851
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2232 rows x 1 columns]

In [360]:
#Llamamos los índices anteriores de Gasto básico y calculamos los índices actuales
gbaant = IndAnt(anio, mes, 'Gba', connection) #este es el que debo modificar para febrero
gbaano = IndAnt(anio,mes, 'Gba', connection, ano = True)
gbaind = pd.DataFrame(gbaant['GbaInd']*vargba['GbaInd'], columns= ['GbaInd']).replace(np.nan, 0)

gbaind #este paso esta de más, es literal lo mismo de arriba solo que en un solo chunk

GbaInd
RegCod GbaCod             
01     0111101  177.796985
       0111201  244.137471
       0111202  238.104250
       0111203  150.948101
       0111301  175.782902
...                    ...
08     1251101  119.542285
       1251102  111.813851
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2232 rows x 1 columns]

Con los índices $I_{GB.r}$ del Gasto básico $GB$ de todas las regiones $r$, podemos calcular el índice del gasto básico a nivel república $I_{GB}$ con la fórmula
$$
I_{GB} = \frac{1}{\sum_{r=1}^8\beta_r \alpha_{GB,r}}\sum_{r=1}^8\beta_r\alpha_{GB,r}I_{GB,r},
$$
donde $\beta_r$ es la ponderación de la región $r$ en el gasto nacional, y $\alpha_{GB,r}$ es la ponderación del gasto básico $GB$ en la región $r$.

In [361]:
#Función para llamar las ponderaciones de cada región
def RegPon(connect):
    """Consulta las ponderaciones asignadas a cada una de las 8 regiones del país

    Args:
    connect (Connection) : una conexión a servidor SQL
    """
    
    query = text(f'SELECT RegCod, RegPon \
        FROM IPC2010_RN.dbo.IPC001 WHERE RegCod != 0')            
    return pd.read_sql(query, connect, index_col= 'RegCod')

#Función para calcular el índice por república
def IndRep(cod :str, index, regpond, indponde):
    ''' 
    Recibe un dataframe de índices por región y retorna el mismo dataframe 

    Args:
    cod (str) : cóigo de la categoría a analizar
    index (DataFrame) : dataframe con los índices, debe tener multiíndice de la forma RegPon,{cod}Cod y el campo {cod}Ind de los índices (o variaciones o contribuciones)
    regpond (DataFrame) : dataframe con las ponderaciones de las 8 regiones
    indpode (DataFrame) : dataframe con las ponderaciones de la categoría
    '''
    indices = index.copy()
    regpon = regpond.copy()
    indpon = indponde.copy()
    indreg = indices.loc[(indices.index.get_level_values(0) != '00')]
    indpond = indpon.loc[(indpon.index.get_level_values(0) != '00')]
    reppon = pd.DataFrame(indpond[cod+'Pon']*regpon['RegPon'],columns = ['Pon'])
    sumas = reppon.groupby(level =cod+ 'Cod').sum()
    regcont = pd.DataFrame(indreg[cod+'Ind']*reppon['Pon']/sumas['Pon'],columns = [cod+'Ind'])
    indrep = regcont.groupby(level = [cod+'Cod']).sum()
    indrep['RegCod'] = '00'
    indrep = indrep.set_index([indrep['RegCod'], indrep.index]).drop(columns = ['RegCod'])
    indices.loc[indices.index.get_level_values(0) == '00'] = indrep
    return pd.concat([indrep, indreg])

In [362]:
#Llamamos las ponderaciones de región y de gastos básicos
regpon = RegPon(connection)
gbapon = CatPon('Gba', di, connection)
GbaAnt = IndRep('Gba', gbaant, regpon, gbapon)
GbaAno = IndRep('Gba', gbaano, regpon, gbapon)

In [363]:
#regpon.to_excel('regpon.xlsx')

In [364]:
#Calculamos los índices a nivel república de gasto básico
GbaInd = IndRep('Gba', gbaind, regpon, gbapon)
GbaInd

GbaInd
RegCod GbaCod             
00     0111101  162.162150
       0111201  225.681735
       0111202  348.136964
       0111203  142.563944
       0111301  192.386161
...                    ...
08     1251101  119.542285
       1251102  111.813851
       1251103  114.954121
       1251104  117.797094
       1251105    0.000000

[2511 rows x 1 columns]

In [365]:
#Nombres(ArtInd,'Art').to_excel('indices_articulos_diciembre_2022.xlsx')

In [366]:
mes

12

In [367]:
Nombres(GbaInd.loc[GbaInd['GbaInd'] != 0],'Gba').sort_values('GbaInd')

GbaNom  \
RegCod GbaCod                                                       
06     0911102  REPRODUCTOR PORTÁTIL DE AUDIO Y VIDEO         ...   
07     0911102  REPRODUCTOR PORTÁTIL DE AUDIO Y VIDEO         ...   
03     0914102  UNIDAD DE RESPALDO MAGNÉTICO                  ...   
08     0911102  REPRODUCTOR PORTÁTIL DE AUDIO Y VIDEO         ...   
06     0921104  CONSOLA DE VIDEOJUEGO                         ...   
...                                                           ...   
02     0116104  NARANJA                                       ...   
       0117118  CULANTRO                                      ...   
04     0117118  CULANTRO                                      ...   
       0117102  GÜISQUIL                                      ...   
       0119203  CHILES PICANTES                               ...   

                      GbaInd  
RegCod GbaCod                 
06     0911102     53.008748  
07     0911102     53.882335  
03     0914102     56.546787  
08     0911102     60.546817  
06     0921104     61.062599  
...                      ...  
02     0116104   7890.914372  
       0117118   8053.288039  
04     0117118   8669.324761  
       0117102  10442.319389  
       0119203  68850.756738  

[2464 rows x 2 columns]

El cálculo del índice $I_{Cat}$ de una categoría $C$ puede realizarse con las ponderaciones e índices de cualquier categoría inferior $c$. 
$$
I_{Cat} = \frac{1}{\alpha_{Cat}} \sum_{c\in Cat} \alpha_{c} I_{c}
$$
donde $\alpha_{Cat} = \sum_{c\in Cat}\alpha_{c}$ es la ponderación de la categoría $Cat$ respecto al gasto nacional, y $\alpha_{c}$ es la ponderación de la categoría $c$ respecto al gasto nacional. Podemos pensar a $c$ como la categoría "pequeña" y a $Cat$ como la categoría "grande" a la que queremos llegar. Esto puede hacerse siempre y cuando la categoría inferior sea igual o superior a Gasto Básico.

In [368]:
Nombres(GbaInd.loc[GbaInd['GbaInd'] != 0],'Gba').sort_values('GbaInd').to_excel('gbaind_enero_2023.xlsx')

In [369]:
''' 
Aquí se pueden calcular los índices de cualquier categoría, sólo se debe cambiar el argumento Div (división) por el argumento Agr, Gru, Sub, Gba dependiendo del índice que se desee
'''
cat = 'Div'
catind = CatInd(cat, GbaInd, gbapon)
catpon = CatPon(cat, di, connection)
catind = IndRep(cat, catind, regpon, catpon)
Nombres(catind, cat)
catind

DivInd
RegCod DivCod            
00     01      280.586140
       02      129.948835
       03      120.257044
       04      129.527068
       05      131.702710
...                   ...
08     08      114.154610
       09      114.183864
       10      106.669690
       11      143.937398
       12      123.344072

[108 rows x 1 columns]

In [370]:
#esto es adicion mia para explorar los datos por division
#dftest = pd.DataFrame(Nombres(catind, cat))
#dftest.to_excel('test.xlsx')

#dftest.loc[dftest['DivNom']=='ALIMENTOS Y BEBIDAS NO ALCOHÓLICAS']
#results = my_dataframe.loc[my_dataframe["column_name"] == my_value]

Por último, para calcular el IPC por región o a nivel nacional, simplemente se hace el promedio ponderado de todas las categorías con su ponderación siempre y cuando sea Gasto Básico o cualquier categoría superior. Es decir, puede calcularse el IPC desde Gasto Básico, Subgrupo, Grupo, Agrupación o División. Esto se hace con la fórmula 
$$
IPC = \frac{1}{\sum_{C}\alpha_{C}}\sum_{C}\alpha_{C}I_{C},
$$
donde $I_{C}$ son los índices de la categoría $C$ y $\alpha_C$ es la ponderación de la categoría $C$ respecto al gasto nacional, la suma es sobre todas las categorías.

Esto es para los índices y ponderaciones a nivel república de las categorías $C$. Si se desea hacer a nivel región $r$, simplemente tomamos los índices y ponderaciones de la región, con la fórmula
$$
IPC_r = \frac{1}{\sum_{Cat}\alpha_{Cat,r}}\sum_{Cat}\alpha_{Cat,r}I_{Cat,r}.
$$

In [371]:
#Función para calcular el IPC
def IPC(index, pond):
    '''
    args:
    index (DataFrame): Conjunto de índices actuales
    pond (DataFrame): Ponderaciones correspondientes al conjunto de índices
    '''
    indices = index.copy()
    indices.columns = ['Ind']
    pon = pond.copy()
    pon.columns = ['Pon']
    return pd.DataFrame(indices['Ind']*pon['Pon']/100, columns = ['IPC']).groupby(level = ['RegCod']).sum()


## Análisis de variaciones
A continuación se analizan las variaciones mensuales e interanuales del IPC para obtener su ritmo inflacionario.

In [372]:
#Calculamos el IPC actual
ipc = IPC(catind, catpon)
ipc

,IPC
RegCod,
00,174.337520
01,143.700283
02,267.153309
03,162.350392
04,247.008137
05,159.393689
06,161.127257
07,241.248471
08,178.991220


In [373]:
#Calculamos el IPC del mes pasado
ipcant = IPC(GbaAnt, gbapon)
ipcant

,IPC
RegCod,
00,174.143173
01,143.447742
02,263.605884
03,162.450621
04,246.170376
05,159.314561
06,160.703493
07,243.571490
08,179.036075


In [374]:
ipcano = IPC(GbaAno, gbapon)
ipcano

,IPC
RegCod,
00,167.346140
01,141.480776
02,247.868296
03,158.647329
04,235.010144
05,154.159447
06,152.777997
07,224.696621
08,170.724664


In [375]:
#Se calculan las variaciones intermensuales
var_mes = (ipc-ipcant)*100/ipcant
var_mes

,IPC
RegCod,
00,0.111602
01,0.176051
02,1.345731
03,-0.061698
04,0.340318
05,0.049668
06,0.263693
07,-0.953732
08,-0.025054


In [376]:
#Se calculan las variaciones interanuales
var_ano = (ipc-ipcano)*100/ipcano
var_ano

,IPC
RegCod,
00,4.177796
01,1.568769
02,7.780347
03,2.334147
04,5.105309
05,3.395343
06,5.464963
07,7.366310
08,4.842040


## Cálculo de Incidencias
Para cuantificar el impacto que tuvo un índice en especícifo $I_C$ en el IPC del mes, podemos hacerlo usando la incidencia $m_{C}$. Ésta puede calcularse de la forma
$$
m_{C} = \frac{\alpha_C (I_{C,ac}-I_{C,hi})}{IPC_{hi}},
$$
donde $I_{C,ac}, I_{C,hi}$ son los índices actual e histórico de la categoría deseada, y $IPC_{hi}$ es el IPC del mes anterior.

In [377]:
def Incidencias(index, indexant, pond,ipcante):
    '''
    args:
    index (DataFrame): Conjunto de índices actuales
    indexant (DataFrame): Conjunto de índices anteriores
    pond (DataFrame): Ponderaciones correspondientes al conjunto de índices
    ipcante (DataFrame): IPC del mes o año anterior. Mes anterior para la incidencia intermensual, año anterior para incidencia anual.
    '''
    pon = pond.copy()
    pon.columns = ['Pon']
    indicesant = indexant.copy()
    indicesant.columns = ['Ind']
    indices = index.copy()
    indices.columns = ['Ind']
    return pd.DataFrame(pon['Pon']*(indices['Ind']-indicesant['Ind'])/ipcante['IPC'],columns= ['Incid'])

## Incidencias División

In [378]:
catant = CatInd('Div', GbaAnt, gbapon)

In [379]:
incid_div = Incidencias(catind, catant, catpon, ipcant)
incidnac_div = incid_div.loc[incid_div.index.get_level_values('RegCod') == '00']

In [380]:
incidenciasnac_div = Nombres(incidnac_div, 'Div')
incidenciasnac_div = incidenciasnac_div.sort_values('Incid')
incidenciasnac_div.sort_index()

DivNom     Incid
RegCod DivCod                                                             
00     01      ALIMENTOS Y BEBIDAS NO ALCOHÓLICAS            ...  0.082345
       02      BEBIDAS ALCOHÓLICAS Y TABACO                  ...  0.000534
       03      PRENDAS DE VESTIR Y CALZADO                   ...  0.008747
       04      VIVIENDA, AGUA, ELECTRICIDAD, GAS Y OTROS COMB... -0.023923
       05      MUEBLES, ARTÍCULOS PARA EL HOGAR Y PARA LA CON... -0.012418
       06      SALUD                                         ...  0.002785
       07      TRANSPORTE                                    ...  0.004187
       08      COMUNICACIONES                                ... -0.003530
       09      RECREACIÓN Y CULTURA                          ...  0.033925
       10      EDUCACIÓN                                     ...  0.000079
       11      RESTAURANTES Y HOTELES                        ...  0.024267
       12      BIENES Y SERVICIOS DIVERSOS                   ... -0.005397

In [381]:
incidencias_div = Nombres(incid_div, 'Div')
incidencias_div = incidencias_div.sort_values('Incid')
incidencias_div

,,DivNom,Incid
RegCod,DivCod,,
07,01,ALIMENTOS Y BEBIDAS NO ALCOHÓLICAS ...,-0.939765
03,05,"MUEBLES, ARTÍCULOS PARA EL HOGAR Y PARA LA CON...",-0.076351
06,07,TRANSPORTE ...,-0.058655
03,01,ALIMENTOS Y BEBIDAS NO ALCOHÓLICAS ...,-0.050105
07,07,TRANSPORTE ...,-0.050100
...,...,...,...
03,11,RESTAURANTES Y HOTELES ...,0.079309
00,01,ALIMENTOS Y BEBIDAS NO ALCOHÓLICAS ...,0.082345
04,01,ALIMENTOS Y BEBIDAS NO ALCOHÓLICAS ...,0.288366


## Incidencias Gastos Básicos

In [382]:
incid_gba = Incidencias(GbaInd, GbaAnt, gbapon, ipcant)
incidnac_gba = incid_gba.loc[incid_gba.index.get_level_values('RegCod') == '00']

In [383]:
incidencias_gba = Nombres(incid_gba, 'Gba').sort_values('Incid')
incidencias_gba = incidencias_gba.loc[incidencias_gba['Incid'].notnull()]
incidencias_gba

GbaNom     Incid
RegCod GbaCod                                                              
02     0111202  MAÍZ                                          ... -0.463205
07     0117102  GÜISQUIL                                      ... -0.415968
04     0117102  GÜISQUIL                                      ... -0.200668
07     0117101  TOMATE                                        ... -0.198181
       0117120  OTRAS LEGUMBRES Y HORTALIZAS                  ... -0.170461
...                                                           ...       ...
04     0117111  CEBOLLA                                       ...  0.237355
01     0732101  SERVICIO DE TRANSPORTE AÉREO                  ...  0.243620
02     0116104  NARANJA                                       ...  0.260969
       0117111  CEBOLLA                                       ...  0.454446
       0117101  TOMATE                                        ...  0.811141

[2464 rows x 2 columns]

In [384]:
incidenciasnac_gba = Nombres(incidnac_gba, 'Gba').sort_values('Incid')
incidenciasnac_gba

GbaNom     Incid
RegCod GbaCod                                                              
00     0117102  GÜISQUIL                                      ... -0.088535
       0722101  GASOLINA SUPERIOR                             ... -0.067489
       0111202  MAÍZ                                          ... -0.056843
       0116101  AGUACATE                                      ... -0.031200
       0442101  GAS PROPANO                                   ... -0.029425
...                                                           ...       ...
       0951102  SERVICIOS DE VIAJES TODO INCLUIDO FUERA DEL PA...  0.022363
       0111501  PRODUCTOS DE TORTILLERÍA                      ...  0.035550
       0732101  SERVICIO DE TRANSPORTE AÉREO                  ...  0.108373
       0117101  TOMATE                                        ...  0.116284
       0117111  CEBOLLA                                       ...  0.120633

[279 rows x 2 columns]

In [385]:
incidenciasnac_gba[:10]

GbaNom     Incid
RegCod GbaCod                                                              
00     0117102  GÜISQUIL                                      ... -0.088535
       0722101  GASOLINA SUPERIOR                             ... -0.067489
       0111202  MAÍZ                                          ... -0.056843
       0116101  AGUACATE                                      ... -0.031200
       0442101  GAS PROPANO                                   ... -0.029425
       0117112  PAPA                                          ... -0.024355
       0722102  GASOLINA REGULAR                              ... -0.023752
       0117113  ZANAHORIA                                     ... -0.015568
       0117118  CULANTRO                                      ... -0.010212
       0711101  AUTOMOVIL Y PICK UP                           ... -0.008193

In [386]:
incidenciasnac_gba[-10:].sort_values(by='Incid', ascending=False)

GbaNom     Incid
RegCod GbaCod                                                              
00     0117111  CEBOLLA                                       ...  0.120633
       0117101  TOMATE                                        ...  0.116284
       0732101  SERVICIO DE TRANSPORTE AÉREO                  ...  0.108373
       0111501  PRODUCTOS DE TORTILLERÍA                      ...  0.035550
       0951102  SERVICIOS DE VIAJES TODO INCLUIDO FUERA DEL PA...  0.022363
       0951101  SERVICIOS DE VIAJES TODO INCLUIDO DENTRO DEL P...  0.016445
       0117103  CHILE PIMIENTO                                ...  0.014756
       1111102  ALMUERZO CONSUMIDO FUERA DEL HOGAR            ...  0.012372
       0112301  CARNE DE POLLO                                ...  0.010958
       1111103  OTRAS COMIDAS CONSUMIDAS  FUERA DEL HOGAR     ...  0.009592

## Incidencias nivel Región

In [387]:
ipcreg = ipc.loc[ipc.index != '00']
ipcantreg = ipcant.loc[ipcant.index != '00']
ipc_ant = ipcant.loc['00','IPC']

In [388]:
incid_reg = pd.DataFrame(regpon['RegPon']*(ipcreg['IPC']-ipcantreg['IPC'])/ipc_ant, columns = ['Incid'])
incid_reg

,Incid
RegCod,
01,0.054162
02,0.110084
03,-0.004280
04,0.031437
05,0.004983
06,0.045097
07,-0.128881
08,-0.001060
